<a href="https://colab.research.google.com/github/mgmeti/projects-/blob/main/Problem_Statement_Santander_Customer_Satisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-danger" style="color:black;background-color:#F6E6E2">

# AI 1: ML/AI Basics
## Project: Santander Customer Satisfaction

**AI 1 Cohort 3**<br/>
**Univ.AI**<br/>
**Instructor**: Pavlos Protopapas<br />

**Team Name:** Stat.<br />
**Team Members:**
Anush Pittu,
Mallikarjun Meti,
Ritik Gupta,
Sibi Chakravarthy Karunakaran.

Links:

1. https://www.kaggle.com/c/santander-customer-satisfaction/data
2. https://edstem.org/us/courses/14588/lessons/29560/slides/169380

## Overview:<br/>

We will be downloading the Santander Customer Satisfaction dataset from Kaggle. The dataset has two different .csv files - training and testing datasets. We will first explore the training data. During pre-processing, we eliminate  a few features which are not significant to the predict the target class. Then, we experiment with fitting logistc regression and decision tree models and then fit ensemble models. Finally, we will compare the performance of all the models.

In [ ]:
#Installing Kaggle API
!pip install --upgrade --force-reinstall --no-deps kaggle

  Using cached kaggle-1.5.16-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [ ]:
!gdown --id 1iZTNwxjrCQ1sKpi9p0GejdVPu2ayMyzH

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1iZTNwxjrCQ1sKpi9p0GejdVPu2ayMyzH
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 325kB/s]


In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Downloading Santander Customer Satisfaction dataset
!kaggle competitions download -c santander-customer-satisfaction

401 - Unauthorized


In [ ]:
#Unzipping the dataset
!unzip /content/santander-customer-satisfaction.zip

unzip:  cannot find or open /content/santander-customer-satisfaction.zip, /content/santander-customer-satisfaction.zip.zip or /content/santander-customer-satisfaction.zip.ZIP.


In [ ]:
#Importing all necessorry libraries
import pandas as pd   # for data analysis and manipulation
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb



# to sample the hyperparameter space based on distributions
from scipy import stats                                                                                                                                                                                                                lol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,k
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

# metrics for performance comparison
from sklearn.metrics import accuracy_score
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score

# ensemble models for classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

import seaborn as sns

plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

<ipython-input-8-71af6d20616a>:32: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
# load the Santander Customer Satisfaction dataset
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# Check number of rows and columns for test and train datasets
print("Train Data Shape : ",train_data.shape)
print("Test Data Shape : ",test_data.shape)

FileNotFoundError: ignored

## Exploratory Data Analysis<br/>


To explore the dataset, we will do the following tasks.<br/>

*   Look into the first 5 rows of the dataframe.<br/>
*   Observe any naming convention used for column names.<br/>
*   Get the summary of the dataframe using .info() <br/>
*   Plot some of the variables and try to understand them.<br/>

In [ ]:
# look into the datasets
train_data.head()       # for train dataset
test_data.head()        # for test dataset

**Observation:** It looks like most of the features are numerical.

In [ ]:
# Looking into the column names
train_data.columns.values

**Observation:**<br/>
we can see three big groups of features:


1.   Features starting with imp_, num_, saldo_. Probably from importe (amount), numerico (numerical), and saldo (balance). They should be numerical.
2.   Features starting with deltaimp, deltanum. A feature linked and probably calculated based on the previous features. They should be numerical too.

3.   Features starting with ind_. Looks like an index (numerical).

<br/> Other than the group of features above, these are the other features in our dataset:<br>

*   var3<br>
*   var15<br>

*   var38<br>
*   TARGET




In [ ]:
# To get an overall summary of the types of the columns in the dataset
train_data.info()

**var3:**<br/>
According to some Kaggle users, the var3 feature would correspond to the customer country. Let's look into this feature.

In [ ]:
# Getting value_counts() for var3
train_data.var3.value_counts()

**Observation:**<br/>
Looks like -999999 is a placeholder when the country is unkown. We'll replace this with the most common value 2 later.

In [ ]:
# filter by top countries, excluding the most common one (2) and see the distribution
top_countries = train_data[(train_data.var3 != -999999) & (train_data.var3 != 2)].groupby('var3').filter(lambda x: len(x) > 80)

# plot number of satisfied (0)/ unsatisfied customers by country
sns.catplot(x='var3', hue='TARGET', kind='count', data=top_countries)


**var15:**<br>
According to some kagglers, var15 feature could correspond to the customer age. Let's explore it.

In [ ]:
# Getting value_counts() for var15
train_data.var15.value_counts()

**Observation:**<br/>
It makes sense looking at the data. The most common age (var15) is 23 years. Let's further see how the age relates to the customer satisfaction.

In [ ]:
# Getting number of rows of data for var15 (age) less than 23
print(train_data[(train_data.var15 < 23)].shape)

# Getting the sum of TARGET for var15 (age) less than 23
print(train_data[(train_data.var15 < 23)].TARGET.sum())

**Observation:**<br/>
There are no unsatisfied customers below 23 years. Let's further look at the satisfaction rate for every age.

In [ ]:
# Plotting TARGET against var15 (age) to look at the satisfaction rate for every age
from matplotlib.ticker import PercentFormatter
g = sns.catplot(x='var15', y='TARGET', kind='bar', data=train_data[(train_data.var15 > 22) & (train_data.var15 < 100)], aspect=3)

for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter())
plt.title('TARGET against var15 (age) to look at the satisfaction rate for every age')
plt.show()

**Observation:**<br/>
In the age group 23 to 40, the percentage of unsatisfied customers increases as age increases. In the age group 41 to 60, the percentage of unsatisfied customers remains fairly constant with age. In the age group 61 to 78, the percentage of unsatisfied customers decreases with age.

**var38:**<br>
According to some users, var38 could correspond to the Mortgage. Let's see.

In [ ]:
# Getting value_counts() for var38
train_data.var38.value_counts()

In [ ]:
# Plotting a boxplot for var38
plt.boxplot(train_data['var38'])
plt.title('Boxplot for var38')
plt.show()

**Observation:**<br/>
The most common value for var38 is 117310, which according to some users may correspond to the median value of a mortgage in Spain. Let's further explore var38 below.

In [ ]:
# Plotting histogram to get the distribution of var38 feature
train_data[(train_data.var38 != 117310.979016494) & (train_data.var38 < 300000)].var38.hist(bins=20)

**Observation:**<br/>
Here we can see that the most common value is actually close to the average of the rest of the mortgages.<br> Let's look at the value of the mortgage of the unsatisfied customers:

In [ ]:
# Plotting histogram to get the distribution of var38 feature for unsatisfied customers
train_data[(train_data.var38 != 117310.979016494) & (train_data.var38 < 300000) & (train_data.TARGET == 1)].var38.hist(bins=20)

**Observation:**<br/>
var38 follows a similar distribution as above for unsatisfied customers, and we don't see a direct relation between the amount of the mortgage and the customer satisfaction.

**TARGET: Customer satisfaction**<br>
As mentioned before, our target feature is the customer satisfaction:<br>
0 for satisfied customers<br>
1 for unsatisfied customers<br>
Let's look at the distribution of the classes.

In [ ]:
# happy customers have TARGET==0, unhappy custormers have TARGET==1
# A little less then 4% are unhappy => unbalanced dataset
pd.DataFrame(train_data['TARGET'].value_counts())

fig, ax = plt.subplots(1,2,figsize=(10,6))
train_data['TARGET'].value_counts().plot.pie(
    explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)

sns.countplot('TARGET', data=train_data, ax=ax[1])
plt.show()

**Observation:**<br/>
Less than 4% of our customers are unsatisfied.<br>
We will probably need to do some resampling (either upsampling or downsampling) to balance the classes. <br>
We'll also need to take this into account while splitting our dataset in train data and test data.

In [ ]:
# checking multicollinearity between the features
features = train_data.drop(['ID','TARGET'],axis=1)

features[features.columns[:8]].corr()


sns.heatmap(features[features.columns[:8]].corr(),annot=True,cmap='YlGnBu')
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.title('Multicollinearity of top 8 features')
plt.show()

**num_var4:**<br/>
According to dmi3kno (see https://www.kaggle.com/cast42/santander-customer-satisfaction/exploring-features/comments#115223),
num_var4 could be the number of bank products. Let's plot the distribution for num_var4.

In [ ]:
# Plotting the distribution of num_var4 (number of bank products)
train_data.num_var4.hist(bins=100)
plt.xlabel('Number of bank products')
plt.ylabel('Number of customers in train')
plt.title('Most customers have 1 product with the bank')
plt.show()

In [ ]:
# Let's look at the density of the of happy/unhappy customers in function of the number of bank products
sns.FacetGrid(train_data, hue="TARGET", size=6) \
   .map(plt.hist, "num_var4") \
   .add_legend()
plt.title('Unhappy cosutomers have less products')

train_data[train_data.TARGET==1].num_var4.hist(bins=6)
plt.title('Amount of unhappy customers in function of the number of products')

plt.show()

##  Preprocessing and Cleaning of the data







In [ ]:
# look into the min, max, mean of the features
train_data.describe()

In [ ]:
# check for null values in data set
null_value = train_data.isnull().sum()
null_value

**Observation:**<br/>
There is no null value in the dataset.

In [ ]:
# drop the 'ID' and 'TARGET' columns from train data set
train = train_data.drop(columns=['ID', 'TARGET'], axis=1)
test = test_data.drop(columns=['ID'], axis=1)

train.shape, test.shape

**Constant Features Removal**<br>
We remove the features which have zero variance.<br>
Constant features provide no information that can help in classification.

In [ ]:
# Feature selector that removes all low-variance features.
constant_filter = VarianceThreshold(threshold=0)
# Train data set
# fit the varaince threshold object
constant_filter.fit(train)
# transform the train data
constant_filter.transform(train)
# get_spport() will give boolean output
# we get train dataframe by removing constant features
train_filter = train[train.columns[constant_filter.get_support()]]
# print the shape of the train data set before and after.
print('Train dtat set:',train_filter.shape,train.shape)

test_filter = test[test.columns[constant_filter.get_support()]]
# print the shape of the test data set before and after.
print('Test data set:',test_filter.shape,test.shape)


**Observation:**<br/>
The number of features has been reduced from 369 to 335, which means 34 constant features have been removed.


**Quasi Constant Features Removal:**<BR/>
Quasi Constant is like constant feature. In constant feature the VarianceThreshold is 0, while in quasi-constant feature  VarianceThreshold is 0.1.


In [ ]:
# Feature selector that removes all low-variance features(threshold=0.01)
quasi_constant_filter = VarianceThreshold(threshold=0.01)

# Train data set
# fit the varaince threshold object
quasi_constant_filter.fit(train_filter)
# transform the train data
quasi_constant_filter.transform(train_filter)
# get_spport() will give boolean output
# we get train dataframe by removing constant features
train_quasi_filter = train_filter[train_filter.columns[quasi_constant_filter.get_support()]]
# print the shape of the train data set before and after.
print('Train data set:',train_quasi_filter.shape, train_filter.shape)


# we get test dataframe by removing constant features
test_quasi_filter = test_filter[test_filter.columns[quasi_constant_filter.get_support()]]
# print the shape of the test data set before and after.
print('Test data set:', test_quasi_filter.shape, test_filter.shape)


**Observation:**<br/>
After the removal of quasi constant features, the number of features has been reduced from 371 to 272.

**Duplicate Features Removal:**<br>
Sometimes we have duplicate columns.<br>
We take the first column and remove the duplicate column.

In [ ]:
# take  transpose of filtered train data set  and test data set
train_Trans = train_quasi_filter.T
test_Trans = test_quasi_filter.T

# convert array into dataframe
X_train_T = pd.DataFrame(train_Trans)
X_test_T = pd.DataFrame(test_Trans)

# print the shape of transposed train data
print('The shape of transposed train data', X_train_T.shape)

# here df.duplicated checks for duplicates along rows( it gives boolean output)
duplicated_features = X_train_T.duplicated()

# print number of duplicates are there in the data set
print('Number of duplicates are ',duplicated_features.sum())

# get the binary list as true for keeping feature in dta set by inverting duplicated_features
features_to_keep = [not index for index in duplicated_features]

# get the unique  data set and transpose back to original state
X_train_unique = X_train_T[features_to_keep].T
# print the shape of the train data set before and after.
print('Train data set:',X_train_unique.shape, train.shape)

# get the unique  data set and transpose back to original state
X_test_unique = X_test_T[features_to_keep].T
# print the shape of the train data set before and after.
print('Test data set:',X_test_unique.shape, test.shape)


**Observation:**<br/>
After the removal of duplicate features, the number of features has been reduced from 272 to 255.

**Correlated Features Removal:**<br>
There can be some columns which have high correlation.<br> We will check correlation between two columns i and j and if the correlation is greater than 0.98, we will remove column i from the dataset.<br> We will repeat this exercise for all columns in the dataset.

In [ ]:
#Defining a correlation function which will return the name of the correlated columns
def correlation(dataset, threshold):
    col_corr = set()   # Set of all the names of correlated columns
    corr_matrix = dataset.corr()   # calculates the correlation
    for i in range(len(corr_matrix.columns)):

        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:  # we check absolute coeff value and  threshold
                colname = corr_matrix.columns[i]  # getting the name of column out of i, j
                col_corr.add(colname)
    return col_corr

In [ ]:
#getting the name of the correlated columns by calling correlation function (threshold=0.98)
corr_features = correlation(X_train_unique, 0.98)

# print number of correlated features
print('The number of correlated features are :', len(set(corr_features)))

# Drop the correlated columns from data set
train_clean = X_train_unique.drop(corr_features,axis=1)

# Drop the correlated columns from data set
test_clean = X_test_unique.drop(corr_features,axis=1)

# print the shape of data set after droping the correalated features
print('Train data set:',train_clean.shape, train.shape)
# print the shape of data set after droping the correalated features
print('Train data set:',test_clean.shape, test.shape)


In [ ]:
# Replace placeholder in the feature var3 (country) with most common term 2
train_clean['var3'] = train_clean['var3'].replace(-999999, 2)

In [ ]:
# creating a copy of train_clean and assigning it to train_df
# Modifications to the data or indices of the copy will not be reflected in the
# original object
train_df  = train_clean.copy()

In [ ]:
# take copy of the test data set
x_test = test_clean.copy()

y_true = pd.read_csv('/content/sample_submission.csv')
print('Y_true has 0 and 1 :',pd.DataFrame(y_true['TARGET'].value_counts()))
y_true = y_true['TARGET']

# print the shape of test data set
print('The shape of the test data',x_test.shape)

In [ ]:
# assigning x and y variables
# train_df is preprocessed data set (columns ID, TAREGET are dropped)
# it has predictores only
x = train_df

# Response variable
y = train_data['TARGET']

In [ ]:
# Split the x, y as 80% train and 20% for validation, Random_state= 0, stratify by response variable
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.2, random_state = 0, stratify = y)

# print the shapes of train, validation after splitting the predictors and response variable
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

**Scaling:**





we will standarize the features in  training and validation datasets using StandardScaler()<br>
We won't standarize the TARGET feature.

In [ ]:
# standardizing features to unit variance
sc = StandardScaler()
# fit the standard scale with x_train
sc.fit(x_train)

#getting standardized x_train
x_train_std = sc.transform(x_train)

#getting standardized x_val
x_val_std = sc.transform(x_val)

#getting standardized x_test
x_test_std = sc.transform(x_test)

**Defining a function to evaluate the classifier:**<br>
We will define a function to evaluate the classifier performance. <br>We will print the confusion matrix, accuracy, precision, recall, F1 score and roc_auc.

In [ ]:
# Function for getting metrices for a classifier with y_test, y_pred and y_pred_proba as arguments
def get_clf_eval(y_test, pred = None, pred_proba = None):
    # for confusion matrix of true and predicted values
    confusion = confusion_matrix(y_test, pred)
    # for accuracy of the model
    accuracy = accuracy_score(y_test, pred)
    # for precision of the model
    precision = precision_score(y_test, pred)
    # for recall of the model
    recall = recall_score(y_test, pred)
    # for f1_score of the model
    f1 = f1_score(y_test, pred)
    # for area under curve of the model
    roc_auc = roc_auc_score(y_test, pred_proba)

    # print all the metrices for given model
    print('Confusion matrix')
    print(confusion)
    print('Accuracy : {}'.format(np.around(accuracy,4)))
    print('Precision: {}'.format(np.around(precision,4)))
    print('Recall : {}'.format(np.around(recall,4)))
    print('F1 : {}'.format(np.around(f1,4)))
    print('ROC_AUC : {}'.format(np.around(roc_auc,4)))

**Defining a function to plot the confusion matrix for a given classifier:**<br>
Plot the confusion matrix with labels for true positive, true negative, false positive and false negative and get the count and percentage for each group.

In [ ]:
# Function for ploting confusion matrix for a classifier
def plot_matrix(cf_matrix):
    # names of groups
    group_names = ['True Neg','False Pos','False Neg','True Pos']

    # number of counts for each group by flattening matrix
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]

    # calculate the percentages of counts for particular group
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]

    # give labels for each group by combining names, count, percentages
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]

    # convert flattened matrix back into the 2X2
    labels = np.asarray(labels).reshape(2,2)
    # plot the heatmap using sns module
    sns.heatmap(cf_matrix, annot=labels, fmt='')

## Base Models<br>
For the base models, we will implement LogisticRegression and DecisionTreeClassifier and we will compare the performance of the models with each other.

####  Logistic Regression<br/>


In [ ]:
# Initiating a logistic regression model
lr_model = LogisticRegression(max_iter = 10000, random_state=0)

# fitting the logistic regression model on x_train_std and y_train
lr_model.fit(x_train_std, y_train)

# predicting the target class for x_val_std
y_pred = lr_model.predict(x_val_std)

# predicting the probabilities for the target class using predict_proba
y_pred_proba = lr_model.predict_proba(x_val_std)[:,1]

# calling the function to evaluate the classifier
get_clf_eval(y_val,y_pred,y_pred_proba)

# plotting confusion_matrix
cf_matrix = confusion_matrix( y_val,y_pred)
plot_matrix(cf_matrix)




**RandomizedSearchCV:**<br/>
Hyperparameter Tuning<br>
We use RandomizedSearchCV to get the best value of C (inverse of regularization strength) for the logistic regression model, to optimize the roc_auc.

In [ ]:
# references: https://www.kaggle.com/solegalli/customer-satisfaction-with-imbalanced-data

# set up the LogisticRegression with default parameters
lr = LogisticRegression(random_state=0)

# determine the hyperparameter space
# make list of c values

param_grid ={"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}

# set up the search
search = RandomizedSearchCV(
    lr,                # the model
    param_grid,         # hyperparam space
    scoring='roc_auc',   # metric to optimize
    cv=5,                 # k-fold cv = 5
    n_iter = 60,
    random_state=0,       # reproducibility
    refit=True,  )

# find best hyperparameters by fitting model
search.fit(x_train_std, y_train)

# the best hyperparameters are stored in an attribute:
print('The best hypermater in given hyper space is :', search.best_params_)

In [ ]:
# getting a new confusion matrix for the model with the best C value
# predict by using best model
y_pred = search.predict(x_val_std)
# get the probabilities of predictors
y_pred_proba = search.predict_proba(x_val_std)[:,1]
# get the metrices of the best model
get_clf_eval(y_val,y_pred,y_pred_proba)
# plot the confusion matrix by using plot confusion matrix
cf_matrix = confusion_matrix( y_val,y_pred)
plot_matrix(cf_matrix)

In [ ]:
# for test data set
y_pred = search.predict(x_test_std)
y_pred_proba = search.predict_proba(x_test_std)[:,1]
# calling the function to evaluate the classifier
# get_clf_eval(y_true,y_pred,y_pred_proba)

# # plotting confusion_matrix
cf_matrix = confusion_matrix( y_true,y_pred)
print('accuracy of test data:',accuracy_score(y_true, y_pred))
plot_matrix(cf_matrix)

### DecisionTreeClassifier
Now, let's use DecisionTreeClassifier on the dataset and compare the performance of the model with that of logistic regression model.

In [ ]:
# take object of decisiontreeclassifier with random state 0 and with default max_depth
dt = DecisionTreeClassifier(random_state=0)
# fit decision tree with train data
dt.fit(x_train_std, y_train)
# predict for validation data
y_pred = dt.predict(x_val_std)
y_pred_proba = dt.predict_proba(x_val_std)[:,1]
# get the metrices for decision tree
get_clf_eval(y_val,y_pred,y_pred_proba)
# plot the confusion matrix
cf_matrix = confusion_matrix( y_val,  y_pred)
plot_matrix(cf_matrix)

**RandomizedSearchCV:**<br/>
We use RandomizedSearchCV to get the best value of max_depth for the DecisionTreeClassifier model, to optimize the roc_auc.

In [ ]:
# set up the DecisionTreeClassifier with default parameters
dt = DecisionTreeClassifier(random_state=0)

# determine the hyperparameter space
# we use stats to sample from distributions
param_grid =dict(max_depth=stats.randint(1, 30))

# set up the search
search = RandomizedSearchCV(
    dt,             # the model
    param_grid,     # hyperparam space
    scoring='roc_auc',  # metric to optimize
    cv=5,
    n_iter = 60,
    random_state=0, # reproducibility
    refit=True)

# find best hyperparameters
search.fit(x_train_std, y_train)

# the best hyperparameters are stored in an attribute:
print('The best hypermater in given hyper space is :', search.best_params_)

In [ ]:
#getting new y_pred, given the best max_depth value as obtained from the RandomizedSearchCV
y_pred = search.predict(x_val_std)
y_pred_proba = search.predict_proba(x_val_std)[:,1]
# get the matrices for best decision tree
get_clf_eval(y_val,y_pred,y_pred_proba)
# plot the confusion matrix
cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)

## Ensemble models for imbalanced data <br>
Having tried logistic regression and decision tree models, now we will implement ensemble models such as GradientBoostingClassifier, BaggingClassifier and RandomForestClassifier and compare the results with the base models.

### GradientBoostingClassifier

In [ ]:
# I use GBM because it usually out-performs other off-the-shelf
# classifiers

# set up the gradient boosting classifier with default parameters
gbm = GradientBoostingClassifier(random_state=0)

# determine the hyperparameter space
# we use stats to sample from distributions

param_grid = dict(
    n_estimators=stats.randint(10, 200),
    min_samples_split=stats.uniform(0, 1),
    max_depth=stats.randint(1, 5),
    loss=('deviance', 'exponential'),
    )

# set up the search
search = RandomizedSearchCV(
    gbm,
    param_grid,
    scoring='roc_auc',
    cv=2,
    n_iter = 5,
    random_state=0,
    refit=True,
)

# find best hyperparameters
search.fit(x_train_std, y_train)

# the best hyperparameters are stored in an attribute:
print('The best hyperparmeters are :',search.best_params_)

# using the best values of hyperparameters on gbm and plotting confusion matrix
y_pred = search.predict(x_val_std)
y_pred_proba = search.predict_proba(x_val_std)[:,1]
# get the metrices for best gbm model
get_clf_eval(y_val,y_pred,y_pred_proba)
# plot the confusion matrix
cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)

In [ ]:
# for test data set
y_pred = search.predict(x_test_std)
y_pred_proba = search.predict_proba(x_test_std)[:,1]
# calling the function to evaluate the classifier
# get_clf_eval(y_true,y_pred,y_pred_proba)

# # plotting confusion_matrix
cf_matrix = confusion_matrix( y_true,y_pred)
print('accuracy of test data:',accuracy_score(y_true, y_pred))
plot_matrix(cf_matrix)

### BaggingClassifier

In [ ]:
# use ensemble bagging to improve the performance of the model
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators = 100, max_features = 0.5)

# find best hyperparameters
clf.fit(x_train_std, y_train)

# the best hyperparameters are stored in an attribute:
print('The best hyperparmeters are :',search.best_params_)

y_pred = clf.predict(x_val_std)
y_pred_proba = clf.predict_proba(x_val_std)[:,1]
get_clf_eval(y_val,y_pred,y_pred_proba)

cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)

### RandomForestClassifier

In [ ]:
# Define a Random Forest classifier all default parameters
random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
# fit the model
random_forest.fit(x_train_std, y_train)

y_pred = random_forest.predict(x_val_std)
y_pred_proba = random_forest.predict_proba(x_val_std)[:,1]
get_clf_eval(y_val,y_pred,y_pred_proba)

cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)


### AdaBoostClassifier

In [ ]:
# used boost ensemble with default parameters
clf = AdaBoostClassifier(n_estimators=100, random_state=0).fit(x_train_std, y_train)

y_pred = clf.predict(x_val_std)
y_pred_proba = clf.predict_proba(x_val_std)[:,1]
get_clf_eval(y_val,y_pred,y_pred_proba)

cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)

### XGBClassifier

In [ ]:
# XGBooost classifier with default parameters
model = XGBClassifier()
model.fit(x_train_std, y_train)

y_pred = model.predict(x_val_std)
y_pred_proba = model.predict_proba(x_val_std)[:,1]
get_clf_eval(y_val,y_pred,y_pred_proba)

cf_matrix = confusion_matrix(y_val,y_pred)
plot_matrix(cf_matrix)

In [ ]:
# for test data set
y_pred = model.predict(x_test_std)
y_pred_proba = model.predict_proba(x_test_std)[:,1]
# calling the function to evaluate the classifier
# get_clf_eval(y_true,y_pred,y_pred_proba)

# # plotting confusion_matrix
cf_matrix = confusion_matrix( y_true,y_pred)
print('accuracy of test data:',accuracy_score(y_true, y_pred))
plot_matrix(cf_matrix)

we used roc_auc as metrice for comparison of models:
out all ensembles models:
XGBoost is giving roc_auc as .8246

we taken thet XGBoost as over best model